## 1. 2D/3D Transformation

### 1.1. 2D Transformation

In [ ]:
import cv2
import matplotlib.pyplot as plt
import math
import numpy as np

You need to implement the function "get_transform_matrix".

In [ ]:
def get_transform_matrix(rotate_angle, scale_factor, center_point):
  """
  Get the transformation matrix (scaling, rotate in center) to transform image
  Inputs:
    rotate_angle: float: rotation angle degree
    scale_factor: int: scale image
    center_points: list: [x, y]: co-ordinate of center point
  Returns:
    The numpy homogeneous transform matrix: (3x3)
  """
  # Need to implement here

You can upload an image, and excecute the 'run' function to test

In [ ]:
def process_image(image, scale_factor=1, rotate_angle=15):
  height, width = image.shape[:2]
  center = (width/2, height/2)
  transform_matrix = get_transform_matrix(rotate_angle, scale_factor, center)
  cv2_wrap_matrix = transform_matrix[:2, :]
  rotated_image = cv2.warpAffine(src=image, M=cv2_wrap_matrix, dsize=(width, height))
  return rotated_image

def show_imgs(rotated_img, orgin_img):
  fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 9))
  axes[0].imshow(orgin_img)
  axes[1].imshow(rotated_img)

  axes[0].set_axis_off()
  axes[1].set_axis_off()

def run(img_path):
  image = cv2.imread(img_path)
  rotated_image = process_image(image)
  show_imgs(rotated_image, image)

In [ ]:
from google.colab import files
# Create a file upload dialog and select the file to upload.
uploaded_file = files.upload()

img_path = next(iter(uploaded_file))
print('Uploaded file path:', img_path)

In [ ]:
run(img_path)

###1.2. 3D Tranformation




In [ ]:
!pip install open3d

In [ ]:
import open3d as o3d

In [ ]:
def load_data():
  bunny = o3d.data.BunnyMesh()
  mesh = o3d.io.read_triangle_mesh(bunny.path)
  mesh.compute_vertex_normals()
  pcl = mesh.sample_points_poisson_disk(number_of_points=10000)
  return pcl


def visualize_result(pcl):
  if isinstance(pcl, np.ndarray):
    point_cloud_o3d = o3d.geometry.PointCloud()
    point_cloud_o3d.points = o3d.utility.Vector3dVector(pcl)
    o3d.visualization.draw_plotly([point_cloud_o3d],
                                up=[0, 1, 0],
                                front=[0, 0, 1],
                                lookat=[0, 0.1, 0],
                                zoom=0.4
                                )
  else:
    o3d.visualization.draw_plotly([pcl],
                                up=[0, 1, 0],
                                front=[0, 0, 1],
                                lookat=[0, 0.1, 0],
                                zoom=0.4
                                )


def rotate_oz_3D_object(point_3d, rotate_angle):
  """
    Rotate object around Oz
    Inputs:
      point_3d: numpy array: shape (N, 3), N is number of points
      rotate_angle: float: rotate angle degree
    Outputs:
      rotated_point_3d: numpy array: shape (N, 3), N is number of points.
  To do:
    1. Create homogeneous rotated matrix (4x4)
    2. Convert point_3d into the homogeneous coordinate
    3. Matrix multiplication to get the result.
  """
  #Need to implement the function

In [ ]:
pcl = load_data()
np_pcl = np.asarray(pcl.points)
rotated_np_pcl = rotate_oz_3D_object(np_pcl, 45)

In [ ]:
visualize_result(np_pcl)

In [ ]:
visualize_result(rotated_np_pcl)

## 2. Camera calibration

In [ ]:
import numpy as np
import cv2
import glob
from pathlib import Path
import os

You need to upload images

In [ ]:
world_points = # need to supply,  numpy array,  need to have shape ((49, 3) if use the chessboard provided)
image_dir_path = # need to change to image dir

In [ ]:
dectected_point_image_path = '/content/results/'
Path(dectected_point_image_path).mkdir(parents=True, exist_ok=True)

In [ ]:
image_points_list = []
world_points_list = []
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
image_path_list = list(Path(image_dir_path).glob("*.jpg"))

for index, image_path in enumerate(image_path_list):
    img = cv2.imread(str(image_path))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        world_points_list.append(world_points)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        image_points_list.append(corners2)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,7), corners2, ret)
        image_path = os.path.join(dectected_point_image_path, f"img_{index}.png")
        cv2.imwrite(str(image_path), img)

In [ ]:
# calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(world_points_list, image_points_list, gray.shape[::-1], None, None)

In [ ]:
print("Intrisic camera matrix: ", np.asarray(mtx).tolist())